# Stellar objects identification with Machine Learning

### Inteligencia Artificial para los Sistemas de Control Autónomos
### Máster de Ciencia y Tecnología desde el Espacio


## Objectives

- Deal with an open Machine Learning problem.
- Classification of tabular unbalanced data.
- Hyperparameter optimization with grid and random search.
- Handle outliers.

## Introduction

The [Sloan Digital Sky Survey](https://www.sdss.org/) (or SDSS) is, according to [Wikipedia](https://en.wikipedia.org/wiki/Sloan_Digital_Sky_Survey), "a major multi-spectral imaging and spectroscopic redshift survey using a dedicated 2.5-m wide-angle optical telescope at Apache Point Observatory in New Mexico, United States". It contains a huge database in which each night hundreds of gigabytes are added.

In this practice we will start from a data set of stellar objects to train a model that identifies the type of object it is from among three categories: Galaxy, star or quasar. The dataset contains 100.000 instances, it is unbalanced and contains a large number of outliers.

The features that the dataset contains are the following ones:

*    *obj_ID* = Object Identifier, the unique value that identifies the object in the image catalog used by the CAS
*    *alpha* = Right Ascension angle (at J2000 epoch)
*    *delta* = Declination angle (at J2000 epoch)
*    *u* = Ultraviolet filter in the photometric system
*    *g* = Green filter in the photometric system
*    *r* = Red filter in the photometric system
*    *i* = Near Infrared filter in the photometric system
*    *z* = Infrared filter in the photometric system
*    *run_ID* = Run Number used to identify the specific scan
*    *rereun_ID* = Rerun Number to specify how the image was processed
*    *cam_col* = Camera column to identify the scanline within the run
*    *field_ID* = Field number to identify each field
*    *spec_obj_ID* = Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)
*    *class* = object class (galaxy, star or quasar object)
*    *redshift* = redshift value based on the increase in wavelength
*    *plate* = plate ID, identifies each plate in SDSS
*    *MJD* = Modified Julian Date, used to indicate when a given piece of SDSS data was taken
*    *fiber_ID* = fiber ID that identifies the fiber that pointed the light at the focal plane in each observation

In [1]:
import pandas as pd

df = pd.read_csv("https://github.com/dfbarrero/dataCourse/raw/master/assignments/stellar/stellar.zip")

df.columns

Index(['obj_ID', 'alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'class', 'redshift',
       'plate', 'MJD', 'fiber_ID'],
      dtype='object')

## Exercise: EDA

In [5]:

# YOUR CODE HERE


## Exercise: Data cleaning

Locate the outliers in the dataset and remove them. There are many methods you can use, one quite easy is to visualize a boxplot or histogram of each attribute and drop those values that are too high or low. You also should remove irrelevant or redundant attributes.

In [6]:

# YOUR CODE HERE


## Dealing with unbalanced classes
There are several actions that can be taken to address an unbalanced dataset, the best approach will depend on the problem and the data itself. 

The classical accuracy used in classification, when there is unbalanced classes, looses reliability. Imagine, for instance, a dataset with 99% of instances of a class A, and only 1% of class B. If we apply a dummy classifier that classifies everything as A, we will obtain an accuracy of 99%, which suggests excellent performance, but clearly this classifier will not be working correctly. For this reason accuracy is rarely used with unbalanced datasets.

One of the most commonly used metrics with unbalanced data is F1, which is defined as the harmonic mean of the precision and recall:

$F1 = 2 \times \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}$.

F1 is defined in a range from zero to one, with one corresponding to a perfect classification. The definition is made for binary classification problems, so it needs to be adapted to multi-label problems like the one we are dealing with. Scikit applies F1 to each class separately, obtaining several F1 values, which it then groups by means of an average. There are several methods to do this grouping. More information in [Scikit-Learn reference documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).

Another approach to deal with an unbalanced dataset is to make it a balanced one. The most straightforward way to this is to simply undersample the majority class or oversample the minority class. A variation of the latter is to add noise to the oversampling in a controlled way, so as to bring more variety to the dataset, avoiding overlearning. In the context of Deep Learning this technique is known as "data augmentation".

<img align="center" src="smote.webp" width="300">

More advanced techniques involve synthetically generating new instances of the minority class. For example, [SMOTE](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/) is a technique of some sophistication and quite popular that creates new instances of the minority class by sampling the straight line joining a minority class instance with some close neighbor. 

SMOTE is not supported by Sckit-Learn, we need another library named imblearn.

In [7]:
!pip install imblearn

/bin/bash: /home/david/anaconda3/envs/ml37/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [8]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE


df = pd.read_csv("https://github.com/dfbarrero/dataCourse/raw/master/assignments/stellar/stellar.zip")
print(df['class'].value_counts())

x = df.drop(['class'], axis = 1)
y = df.loc[:,'class'].values

GALAXY    59445
STAR      21594
QSO       18961
Name: class, dtype: int64


In [9]:
smote= SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)

print(np.unique(y_smote, return_counts=True))

del x_smote, y_smote
del df, x, y

(array(['GALAXY', 'QSO', 'STAR'], dtype=object), array([59445, 59445, 59445]))


## Exercise: Predictive modeling

Train a classifier to identify if a certain object is a galaxy, quasar or star. You can use any technique of your choice to deal with the unbalanced dataset and try, at least, five classifiers. Obtain the best classifier you can without sophisticated hyperparameter optimization.

In [10]:

# YOUR CODE HERE


## Hyperparameter optimization

The performance of the models depends strongly on the hyperparameters, which is more clear when dealing with real-world problems. Since performance can be quantified, we can view this problem as a pure optimization problem, for which there are a multitude of techniques in AI. 

<center> Grid search <img align="center" src="grid.png" width="300"></center>

One of the most widely used hyperparameter optimization techniques in Machine Learning is provided by the Scikit-Learn function *GridSearchCV()*, which is provided with the hyperparameters to be optimized and a range of values to be searched. The function creates a combination of hyperparameters and trains the model with each combination, returning the combination that performs best. It can apply cross-validation if required to.

In [9]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris()

parameters = {'kernel':('linear', 'rbf'), 
              'C':[1, 10]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)

print(f"Best params: {clf.best_params_}")
print(f"Best score: {clf.best_score_}")

Best params: {'C': 1, 'kernel': 'linear'}
Best score: 0.9800000000000001


Please observe that [*GridSearchCV()*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) returns an object that contains the result of the search, but behaves like a model.

<center>Random search <img align="center" src="random.png" width="300"></center>

Since the search space increases exponentially with the number of hyperparameters to be optimized, grid search becomes computationally intractable with some ease. A randomized search may be useful in these cases ([see *RandomizedSearchCV()*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)), or other much more advanced methods such as bayesian optimization or even Genetic Algorithms.

## Exercise: Predictive modeling with hyperparameter optimization

Get the best classifier you can.

In [ ]:

# YOUR CODE HERE
